In [1]:
from load_data import CholeskyStockDataset

zip_path = "../../data/Data_Cleaned.zip"
data_path = "../../data/Data_60min_cleaned"
N_FIRST_STOCKS = 30
TIME_STEPS = 7
PREDICTION_HORIZON = 1
ALPHA = 0.1

dataset = CholeskyStockDataset(
    zip_path,
    data_path,
    sequence_length=TIME_STEPS,
    prediction_horizon=PREDICTION_HORIZON,
    n_first_stocks=N_FIRST_STOCKS,
)

sample_item = dataset[0]
cholesky_vector_size = sample_item[1].shape[1]
print(f"\nLength of dataset:                  {len(dataset)}")
print(f"Shape of eod_data_batch:            {sample_item[0].shape}")
print(f"Shape of cholesky_vectors_batch:    {sample_item[1].shape}")
print(f"Shape of mask_batch:                {sample_item[2].shape}")
print(f"Shape of base_batch:                {sample_item[3].shape}")
print(f"Shape of ground_truth_batch:        {sample_item[4].shape}")

Successfully loaded AAPL with 14084 rows
Successfully loaded AMGN with 14084 rows
Successfully loaded AXP with 14084 rows
Successfully loaded BA with 14084 rows
Successfully loaded CAT with 14084 rows
Successfully loaded CRM with 14084 rows
Successfully loaded CSCO with 14084 rows
Successfully loaded CVX with 14084 rows
Successfully loaded DIA with 14084 rows
Successfully loaded DIS with 14084 rows
Successfully loaded GS with 14084 rows
Successfully loaded HD with 14084 rows
Successfully loaded HON with 14084 rows
Successfully loaded IBM with 14084 rows
Successfully loaded INTC with 14084 rows
Successfully loaded JNJ with 14084 rows
Successfully loaded JPM with 14084 rows
Successfully loaded KO with 14084 rows
Successfully loaded MCD with 14084 rows
Successfully loaded MMM with 14084 rows
Successfully loaded MRK with 14084 rows
Successfully loaded MSFT with 14084 rows
Successfully loaded NKE with 14084 rows
Successfully loaded PG with 14084 rows
Successfully loaded TRV with 14084 rows


In [2]:
from load_data import create_dataloaders

train_loader, valid_loader, test_loader = create_dataloaders(
    dataset,
    train_ratio=0.8,
    valid_ratio=0.1,
    test_ratio=0.1,
    batch_size=1
)

sample_batch = next(iter(train_loader))
eod_data_batch, cholesky_vectors_batch, mask_batch, base_batch, ground_truth_batch = sample_batch

print(f"\nSample batch shapes:")
print(f"Shape of eod_data_batch:            {eod_data_batch.shape}")
print(f"Shape of cholesky_vectors_batch:    {cholesky_vectors_batch.shape}")
print(f"Shape of mask_batch:                {mask_batch.shape}")
print(f"Shape of base_batch:                {base_batch.shape}")
print(f"Shape of ground_truth_batch:        {ground_truth_batch.shape}")



Sample batch shapes:
Shape of eod_data_batch:            torch.Size([1, 30, 7, 4])
Shape of cholesky_vectors_batch:    torch.Size([1, 7, 465])
Shape of mask_batch:                torch.Size([1, 30])
Shape of base_batch:                torch.Size([1, 30])
Shape of ground_truth_batch:        torch.Size([1, 30])


In [3]:
from model import CholeskyStockMixer
from train import test_model
import torch


model = CholeskyStockMixer(
    stocks=N_FIRST_STOCKS,
    time_steps=TIME_STEPS,
    channels=4,
    market=20,
    scale_dim=3,
    cholesky_hidden_size=465,
    cholesky_vector_size=cholesky_vector_size,
    lstm_model_path="../../src/LSTM/model/cholesky_lstm_model_eliot.pth",
    # dropout_rate=0.2
)
model.load_state_dict(torch.load("./model/model_weights.pth"))
history = test_model(model, test_loader=test_loader, stock_num=N_FIRST_STOCKS, alpha=ALPHA, desc="Testing")

{'input_size': 465, 'hidden_size': 465, 'n_assets': 30, 'sequence_length': 7, 'learning_rate': 1e-05, 'lambda_penalty': 5e-07}
Loading trained CholeskyLSTM with hyperparameters:
Input size:     465
Hidden size:    465
N_assets:       30


/home/ryounis/Documents/Zurich/ETHZ/MA3/DeepL/DL_2024/LSTM_Stockmixer/src/model.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_info = torch.load(lstm_model_path)

tensor([ 0.0118, -0.0174, -0.0042, -0.0136, -0.0142, -0.0032, -0.0032, -0.0155,
        -0.0049, -0.0045, -0.0056, -0.0054, -0.0079, -0.0054, -0.0057, -0.0090,
        -0.0061,  0.0012, -0.0032, -0.0053, -0.0118, -0.0004,  0.0027, -0.0024,
        -0.0049, -0.0004,  0.0012, -0.0063, -0.0064,  0.0057],
       dtype=torch.float64)
tensor([ 0.0033,  0.0139,  0.0017,  0.0094,  0.0063,  0.0063,  0.0066,  0.0101,
         0.0047,  0.0024,  0.0029,  0.0033,  0.0056,  0.0051,  0.0063,  0.0056,
         0.0024, -0.0005,  0.0038,  0.0050,  0.0133,  0.0084, -0.0010,  0.0028,
         0.0003,  0.0048,  0.0036,  0.0052,  0.0039, -0.0031],
       dtype=torch.float64)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=torch.float64)


Testing: 100%|██████████| 1409/1409 [00:15<00:00, 88.38it/s] 


In [4]:
history

({'total_loss': 0.00019076979670732007,
  'reg_loss': 0.00018588384771983996,
  'rank_loss': 4.885948987479638e-05},
 {'mse': np.float64(0.00018601577380054975),
  'IC': np.float64(-0.6747467987892735),
  'RIC': np.float64(-2.9324431264639244),
  'sharpe5': np.float64(-23.52760041441193),
  'prec_10': np.float64(0.2995031937544358)})